# Installations/Imports

In [2]:
!pip install openai
!pip install jsonlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [3]:
import os
import requests
import re
from google.colab import drive

import openai
import pandas as pd
import time
from openai import OpenAI

import json
import jsonlines
import csv

from tabulate import tabulate

import nltk
from nltk.tokenize import sent_tokenize

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Mandatory stuff

In [6]:
client = OpenAI(
    api_key = "secret_filler"
)

## Needed functions

In [7]:
def chat_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response

In [8]:
def get_yes_no(gpt_response):
    try:
        response_text = gpt_response.choices[0].message.content.strip().lower()
    except AttributeError:
        response_text = gpt_response.strip().lower()

    if 'yes' in response_text or 'true' in response_text or 'correct' in response_text:
        return 'yes'
    elif 'no' in response_text or 'false' in response_text  or 'incorrect' in response_text:
        return 'no'
    return 'N/A'

In [9]:
def get_difficulty(gpt_response):
    try:
        response_text = gpt_response.choices[0].message.content.strip().lower()
    except AttributeError:
        response_text = gpt_response.strip().lower()

    if '2' in response_text or 'two' in response_text:
        return '2'
    elif '3' in response_text or 'three' in response_text:
        return '3'
    elif '4' in response_text or 'four' in response_text:
        return '4'
    elif '5' in response_text or 'five' in response_text:
        return '5'
    elif '6' in response_text or 'six' in response_text:
        return '6'
    elif '7' in response_text or 'seven' in response_text:
        return '7'
    elif '8' in response_text or 'eight' in response_text:
        return '8'
    elif '9' in response_text or 'nine' in response_text:
        return '9'
    elif ('10' in response_text and ' 1 ' not in response_text) or 'ten' in response_text:
        return '10'
    elif '1' in response_text or 'one' in response_text:
        return '1'
    return 'N/A'

In [10]:
def sanitize_filename(name):
    valid_chars = '-_.() abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
    sanitized_name = ''.join(c if c in valid_chars else '_' for c in name)
    sanitized_name = sanitized_name.strip('_')
    return sanitized_name

# Unnecessary stuff/information

In [ ]:
#'secret_filler'

## Testing

In [ ]:
impersonation = ["You are a preschooler.\n",
                 "You are in elementary school.\n",
                 "You are in high school.\n",
                 "You have a PhD in mathematics.\n"
                 ]

In [35]:
problem_prompt = "What is your name?"

gpt_response = chat_gpt(problem_prompt)

gpt_response_text = gpt_response.choices[0].message.content.strip()

print(gpt_response_text)

I am an AI virtual assistant and I do not have a personal name. You can simply call me Assistant. How can I assist you today?


In [ ]:
solution = "The given expression is the expansion of $(9+1)^3$.  In general, the cube of $(x+y)^3$ is \\[(x+y)^3=1x^3+3x^2y+3xy^2+1y^3.\\]   The first and last terms in the given expression are cubes and the middle two terms both have coefficient 3, giving us a clue that this is a cube of a binomial and can be written in the form \\[(x+y)^3\\]In this case, $x=9$ and $y=1$, so our answer is\\[(9+1)^3\\ = 10^3 = \\boxed{1000}\\]"
pattern = r'\\boxed{([^}]*)}'

match = re.search(pattern, solution)
if match:
    solution_value = match.group(1)

    solution_prompt = "Yes or no, is the final solution obtained in the following equal to " + str(solution_value) + "?:\n" + gpt_response_text
    gpt_response_correct_incorrect = chat_gpt(solution_prompt)
else:
    print("No text inside \\boxed{} found in the solution string.")

# MATH

## Prompt Prefixes

In [ ]:
impersonation = ["Approach the following question the way someone in preschool would. ",
                 "Approach the following question the way someone in elementary school would. ",
                 "Approach the following question the way someone in high school would. ",
                 "Approach the following question the way someone with a PhD in mathematics would. "
                 ]

## Processing

In [9]:
parent_directory = "/content/drive/My Drive/Research/Meeting Prep/6Dataset 1/MATHByLevel/train/number_theory"
output_base_directory = "/content/drive/My Drive/Research/Multi Agent/MATHByLevel/train/number_theory"

pattern = r'\\boxed{([^}]*)}'
os.makedirs(output_base_directory, exist_ok=True)

In [10]:
def process_file(input_file_path, output_directory):

    filename = os.path.basename(input_file_path)
    output_directory_name = os.path.splitext(filename)[0]
    output_directory = os.path.join(output_directory, output_directory_name)
    os.makedirs(output_directory, exist_ok=True)

    json_files_count = len([filename for filename in os.listdir(output_directory) if filename.endswith(".json")])
    if json_files_count == len(impersonation):
        print(f"Skipping {output_directory} (already complete)")
        return

    with open(input_file_path, 'r') as file:
        data = json.load(file)

        problem_prompt = data.get('problem')
        solution = data.get('solution')

        for prefix in impersonation:
            problem_prompt_with_prefix = prefix + problem_prompt

            match = re.search(pattern, solution)

            difficulty_prompt = "On a scale of 1-10, how difficult is this problem? Respond with just the number.: " + problem_prompt
            gpt_response_difficulty = chat_gpt(difficulty_prompt)
            difficulty_value = get_difficulty(gpt_response_difficulty)

            if match:
                for i in range(num_attempts):
                    attempts = i + 1
                    gpt_response = chat_gpt(problem_prompt_with_prefix)
                    solution_value = match.group(1)
                    solution_prompt = "The following is a response to a question. Yes or no, is the final solution obtained in the following response " + str(solution_value) + "?: " + gpt_response.choices[0].message.content.strip()
                    gpt_response_correct_incorrect = chat_gpt(solution_prompt)
                    yes_no = get_yes_no(gpt_response_correct_incorrect)
                    if yes_no == 'yes':
                        break;

                name = prefix[:prefix.rfind(".")]
                output_filename = f"{name}.json"
                output_file_path = os.path.join(output_directory, output_filename)
                with open(output_file_path, 'w') as output_file:
                    json.dump({
                        'prefix': prefix,
                        'problem_prompt': problem_prompt_with_prefix,
                        'solution': solution,
                        'gpt_response': str(gpt_response.choices[0].message.content.strip()),
                        'solution_prompt': solution_prompt,
                        'gpt_response_correct_incorrect': str(gpt_response_correct_incorrect.choices[0].message.content.strip()),
                        'attempts': attempts,
                        'yes_no': yes_no,
                        'gpt_response_difficulty':str(gpt_response_difficulty.choices[0].message.content.strip()),
                        'difficulty_value': difficulty_value
                    }, output_file, indent=4)

            else:
                print("No text inside \\boxed{} found in the solution string:", output_directory)

def process_directory(input_directory, output_base_directory):
    output_directory = os.path.join(output_base_directory, os.path.basename(input_directory))
    os.makedirs(output_directory, exist_ok=True)

    counter = 1
    for filename in os.listdir(input_directory):
        if filename.endswith(".json"):
            input_file_path = os.path.join(input_directory, filename)
            process_file(input_file_path, output_directory)
            print(f'{counter}/{len(os.listdir(input_directory))}')
            counter += 1

In [20]:
num_attempts = 3
level = 1

for directory in os.listdir(parent_directory):
    input_directory = os.path.join(parent_directory, directory)

    print("Level:", level)
    if os.path.isdir(input_directory):
        process_directory(input_directory, output_base_directory)

    level += 1

print("Done")

Level: 1
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel/train/number_theory/Level_1/507 (already complete)
1/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel/train/number_theory/Level_1/1054 (already complete)
2/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel/train/number_theory/Level_1/446 (already complete)
3/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel/train/number_theory/Level_1/15 (already complete)
4/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel/train/number_theory/Level_1/347 (already complete)
5/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel/train/number_theory/Level_1/49 (already complete)
6/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel/train/number_theory/Level_1/722 (already complete)
7/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel/train/number_theory/Level_1/63 (already complete)
8/47
Skipping /content

## Interpreting Results with CSV

### Generate CSV

In [17]:
directory_path = "/content/drive/My Drive/Research/Multi Agent/MATHByLevel/train/number_theory/Level_5"
csv_file_path = "/content/drive/My Drive/Research/Multi Agent/level_5_number_theory_data.csv"

data = []
json_files = ["Approach the following question the way someone in preschool would.json",
              "Approach the following question the way someone in elementary school would.json",
              "Approach the following question the way someone in high school would.json",
              "Approach the following question the way someone with a PhD in mathematics would.json"
              ]

for dir_name in os.listdir(directory_path):
    dir_path = os.path.join(directory_path, dir_name)

    if os.path.isdir(dir_path):
        directory_data = {'Directory Name': int(dir_name)}

        for json_file in json_files:
            json_file_path = os.path.join(dir_path, json_file)
            try:
                with open(json_file_path, 'r') as f:
                    json_data = json.load(f)
                    directory_data[json_file] = json_data['yes_no']
            except FileNotFoundError:
                directory_data[json_file] = 'N/A'

        data.append(directory_data)

sorted_data = sorted(data, key=lambda x: x['Directory Name'])

with open(csv_file_path, 'w', newline='') as csvfile:
    fieldnames = ['Directory Name'] + json_files
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for row in sorted_data:
        writer.writerow(row)

print("Done:", csv_file_path)

Done: /content/drive/My Drive/Research/Multi Agent/level_4_number_theory_data.csv


### Analyze CSV

In [6]:
csv_file_paths = ["/content/drive/My Drive/Research/Multi Agent/level_1_number_theory_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/level_2_number_theory_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/level_3_number_theory_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/level_4_number_theory_data.csv"
                  ]

for csv_file_path in csv_file_paths:
    print(csv_file_path)

    unique_combinations = {}

    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)

        for row in reader:
            row_values = row[1:]

            combination = ''.join([val[:1] for val in row_values])

            unique_combinations[combination] = unique_combinations.get(combination, 0) + 1

    table_data = []
    total_count = 0
    for combination, count in unique_combinations.items():
        preschool = combination[0]
        elementary = combination[1]
        high = combination[2]
        phd = combination[3]
        table_data.append([preschool, elementary, high, phd, count])
        total_count += count

    table_data.append(["Total", "", "", "", total_count])

    print(tabulate(table_data, headers=['Preschool', 'Elementary', 'High', 'PhD', 'Count']))
    print("-------------------------------------------------\n")

/content/drive/My Drive/Research/Multi Agent/level_1_number_theory_data.csv
Preschool    Elementary    High    PhD      Count
-----------  ------------  ------  -----  -------
y            n             y       y            3
y            y             y       y           31
y            n             n       y            1
n            y             y       y            6
y            y             y       n            1
y            y             n       y            1
n            n             y       n            1
n            y             n       y            1
n            n             y       y            1
y            n             y       n            1
Total                                          47
-------------------------------------------------

/content/drive/My Drive/Research/Multi Agent/level_2_number_theory_data.csv
Preschool    Elementary    High    PhD      Count
-----------  ------------  ------  -----  -------
n            y             y       y           

# MMLU

## Prompt Prefixes

In [ ]:
impersonation = ["Approach the following question the way someone in preschool would. ",
                 "Approach the following question the way someone in elementary school would. ",
                 "Approach the following question the way someone in high school would. ",
                 "Approach the following question the way someone with a PhD in mathematics would. "
                 ]

## Processing

In [1]:
def process_csv(file_path, output_directory):
    # Open the CSV file
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)

        row_total = 0
        for row in reader:
            row_total += 1

        csvfile.seek(0)
        # Iterate over each row in the CSV file
        for row_num, row in enumerate(reader):
            row_output_directory = os.path.join(output_directory, str(row_num))
            if not os.path.exists(row_output_directory):
                os.makedirs(row_output_directory)

            json_files_count = len([filename for filename in os.listdir(row_output_directory) if filename.endswith(".json")])
            if json_files_count == len(impersonation):
                print(f"Skipping {row_output_directory} (already complete)")
                continue


            # Extracting the values from each row
            question = row[0]
            choices = row[1:5]
            solution = row[5]

            # Do something with the extracted values
            # print("Question:", question)
            # print("Choices:", choices)
            # print("Solution:", solution)
            # print("\n")

            options = "\nA: " + choices[0] + "\nB: " + choices[1] + "\nC: " + choices[2] + "\nD: " + choices[3]


            #Below is largely copy pasted from MATH above
            problem_prompt = question + options
            solution = solution

            for prefix in impersonation:
                problem_prompt_with_prefix = prefix + problem_prompt

                difficulty_prompt = "On a scale of 1-10, how difficult is this problem? Respond with just the number.: " + problem_prompt
                gpt_response_difficulty = chat_gpt(difficulty_prompt)
                difficulty_value = get_difficulty(gpt_response_difficulty)

                for i in range(num_attempts):
                    attempts = i + 1
                    gpt_response = chat_gpt(problem_prompt_with_prefix)
                    solution_value = solution
                    solution_prompt = "The following is a response to a question. Yes or no, is the final solution obtained in the following response " + str(solution_value) + "?: " + gpt_response.choices[0].message.content.strip()
                    gpt_response_correct_incorrect = chat_gpt(solution_prompt)
                    yes_no = get_yes_no(gpt_response_correct_incorrect)
                    if yes_no == 'yes':
                        break;

                name = prefix[:prefix.rfind(".")]
                output_filename = f"{name}.json"
                output_file_path = os.path.join(row_output_directory, output_filename)

                with open(output_file_path, 'w') as output_file:
                    json.dump({
                        'prefix': prefix,
                        'problem_prompt': problem_prompt_with_prefix,
                        'solution': solution,
                        'gpt_response': str(gpt_response.choices[0].message.content.strip()),
                        'solution_prompt': solution_prompt,
                        'gpt_response_correct_incorrect': str(gpt_response_correct_incorrect.choices[0].message.content.strip()),
                        'attempts': attempts,
                        'yes_no': yes_no,
                        'gpt_response_difficulty':str(gpt_response_difficulty.choices[0].message.content.strip()),
                        'difficulty_value': difficulty_value
                    }, output_file, indent=4)

            print(f'{row_num}/{row_total}')

In [10]:
num_attempts = 3

file_path = "/content/drive/My Drive/Research/Multi Agent/Datasets/MMLU/data/test/college_physics_test.csv"
output_directory = "/content/drive/My Drive/Research/Multi Agent/MMLUEvaluation/college_physics"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

process_csv(file_path, output_directory)

print("Done")

0/102
1/102
2/102
3/102
4/102
5/102
6/102
7/102
8/102
9/102
10/102
11/102
12/102
13/102
14/102
15/102
16/102
17/102
18/102
19/102
20/102
21/102
22/102
23/102
24/102
25/102
26/102
27/102
28/102
29/102
30/102
31/102
32/102
33/102
34/102
35/102
36/102
37/102
38/102
39/102
40/102
41/102
42/102
43/102
44/102
45/102
46/102
47/102
48/102
49/102
50/102
51/102
52/102
53/102
54/102
55/102
56/102
57/102
58/102
59/102
60/102
61/102
62/102
63/102
64/102
65/102
66/102
67/102
68/102
69/102
70/102
71/102
72/102
73/102
74/102
75/102
76/102
77/102
78/102
79/102
80/102
81/102
82/102
83/102
84/102
85/102
86/102
87/102
88/102
89/102
90/102
91/102
92/102
93/102
94/102
95/102
96/102
97/102
98/102
99/102
100/102
101/102
Done


## Interpreting Results with CSV

### Generate CSV

In [11]:
directory_path = "/content/drive/My Drive/Research/Multi Agent/MMLUEvaluation/college_physics"
csv_file_path = "/content/drive/My Drive/Research/Multi Agent/MMLU_college_physics_data.csv"

data = []
json_files = ["Approach the following question the way someone in preschool would.json",
              "Approach the following question the way someone in elementary school would.json",
              "Approach the following question the way someone in high school would.json",
              "Approach the following question the way someone with a PhD in mathematics would.json"
              ]

for dir_name in os.listdir(directory_path):
    dir_path = os.path.join(directory_path, dir_name)

    if os.path.isdir(dir_path):
        directory_data = {'Directory Name': int(dir_name)}

        for json_file in json_files:
            json_file_path = os.path.join(dir_path, json_file)
            try:
                with open(json_file_path, 'r') as f:
                    json_data = json.load(f)
                    directory_data[json_file] = json_data['yes_no']
            except FileNotFoundError:
                directory_data[json_file] = 'N/A'

        data.append(directory_data)

sorted_data = sorted(data, key=lambda x: x['Directory Name'])

with open(csv_file_path, 'w', newline='') as csvfile:
    fieldnames = ['Directory Name'] + json_files
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for row in sorted_data:
        writer.writerow(row)

print("Done:", csv_file_path)

Done: /content/drive/My Drive/Research/Multi Agent/MMLU_college_physics_data.csv


### Analyze CSV

In [12]:
csv_file_paths = ["/content/drive/My Drive/Research/Multi Agent/MMLU_college_mathematics_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/MMLU_abstract_algebra_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/MMLU_college_physics_data.csv"
                  ]

for csv_file_path in csv_file_paths:
    print(csv_file_path)

    unique_combinations = {}

    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)

        for row in reader:
            row_values = row[1:]

            combination = ''.join([val[:1] for val in row_values])

            unique_combinations[combination] = unique_combinations.get(combination, 0) + 1

    table_data = []
    total_count = 0
    for combination, count in unique_combinations.items():
        preschool = combination[0]
        elementary = combination[1]
        high = combination[2]
        phd = combination[3]
        table_data.append([preschool, elementary, high, phd, count])
        total_count += count

    table_data.append(["Total", "", "", "", total_count])

    print(tabulate(table_data, headers=['Preschool', 'Elementary', 'High', 'PhD', 'Count']))
    print("-------------------------------------------------\n")

/content/drive/My Drive/Research/Multi Agent/MMLU_college_mathematics_data.csv
Preschool    Elementary    High    PhD      Count
-----------  ------------  ------  -----  -------
y            y             y       y           56
y            n             y       y            8
n            y             y       y           15
n            n             y       y            5
y            y             n       n            1
y            y             y       n            3
n            y             n       y            5
N            y             y       n            1
y            y             n       y            2
n            y             y       n            2
n            n             n       y            1
y            n             n       y            1
Total                                         100
-------------------------------------------------

/content/drive/My Drive/Research/Multi Agent/MMLU_abstract_algebra_data.csv
Preschool    Elementary    High    PhD      

# MMLU With Domain Experts

## Prompt Prefixes

In [9]:
impersonation = ["Approach the following question the way someone with a PhD in mathematics would. ",
                 "Approach the following question the way someone with a PhD in physics would. ",
                 "Approach the following question the way someone with a PhD in engineering would. ",
                 "Approach the following question the way someone with a PhD in computer science would. ",
                 "Approach the following question the way someone with a PhD in biology would. ",
                 "Approach the following question the way someone with a PhD in literature would. ",
                 "Approach the following question the way someone with a PhD in philosophy would. "
                 ]

## Processing

In [10]:
def process_csv(file_path, output_directory):
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)

        row_total = 0
        for row in reader:
            row_total += 1

        csvfile.seek(0)
        for row_num, row in enumerate(reader):
            row_output_directory = os.path.join(output_directory, str(row_num))
            if not os.path.exists(row_output_directory):
                os.makedirs(row_output_directory)

            json_files_count = len([filename for filename in os.listdir(row_output_directory) if filename.endswith(".json")])
            if json_files_count == len(impersonation):
                print(f"Skipping {row_output_directory} (already complete)")
                continue


            # Extracting the values from each row
            question = row[0]
            choices = row[1:5]
            solution = row[5]

            options = "\nA: " + choices[0] + "\nB: " + choices[1] + "\nC: " + choices[2] + "\nD: " + choices[3]

            problem_prompt = question + options
            solution = solution

            for prefix in impersonation:
                problem_prompt_with_prefix = prefix + problem_prompt

                difficulty_prompt = "On a scale of 1-10, how difficult is this problem? Respond with just the number.: " + problem_prompt
                gpt_response_difficulty = chat_gpt(difficulty_prompt)
                difficulty_value = get_difficulty(gpt_response_difficulty)

                for i in range(num_attempts):
                    attempts = i + 1
                    gpt_response = chat_gpt(problem_prompt_with_prefix)
                    solution_value = solution
                    solution_prompt = "The following is a response to a question. Yes or no, is the final solution obtained in the following response " + str(solution_value) + "?: " + gpt_response.choices[0].message.content.strip()
                    gpt_response_correct_incorrect = chat_gpt(solution_prompt)
                    yes_no = get_yes_no(gpt_response_correct_incorrect)
                    if yes_no == 'yes':
                        break;

                name = prefix[:prefix.rfind(".")]
                output_filename = f"{name}.json"
                output_file_path = os.path.join(row_output_directory, output_filename)

                with open(output_file_path, 'w') as output_file:
                    json.dump({
                        'prefix': prefix,
                        'problem_prompt': problem_prompt_with_prefix,
                        'solution': solution,
                        'gpt_response': str(gpt_response.choices[0].message.content.strip()),
                        'solution_prompt': solution_prompt,
                        'gpt_response_correct_incorrect': str(gpt_response_correct_incorrect.choices[0].message.content.strip()),
                        'attempts': attempts,
                        'yes_no': yes_no,
                        'gpt_response_difficulty':str(gpt_response_difficulty.choices[0].message.content.strip()),
                        'difficulty_value': difficulty_value
                    }, output_file, indent=4)

            print(f'{row_num}/{row_total}')

In [36]:
num_attempts = 3

file_path = "/content/drive/My Drive/Research/Multi Agent/Datasets/MMLU/data/test/philosophy_test.csv"
output_directory = "/content/drive/My Drive/Research/Multi Agent/MMLUEvaluationDomainExperts/philosophy"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

process_csv(file_path, output_directory)

print("Done")

Skipping /content/drive/My Drive/Research/Multi Agent/MMLUEvaluationDomainExperts/philosophy/0 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/MMLUEvaluationDomainExperts/philosophy/1 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/MMLUEvaluationDomainExperts/philosophy/2 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/MMLUEvaluationDomainExperts/philosophy/3 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/MMLUEvaluationDomainExperts/philosophy/4 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/MMLUEvaluationDomainExperts/philosophy/5 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/MMLUEvaluationDomainExperts/philosophy/6 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/MMLUEvaluationDomainExperts/philosophy/7 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/MMLUEvaluationDomainExperts/philos

## Interpreting Results with CSV

### Generate CSV

In [ ]:
directory_path = "/content/drive/My Drive/Research/Multi Agent/MMLUEvaluationDomainExperts/philosophy"
csv_file_path = "/content/drive/My Drive/Research/Multi Agent/MMLU_philosophy_domain_experts_data.csv"

data = []
json_files = ["Approach the following question the way someone with a PhD in mathematics would.json",
               "Approach the following question the way someone with a PhD in physics would.json",
               "Approach the following question the way someone with a PhD in engineering would.json",
               "Approach the following question the way someone with a PhD in computer science would.json",
               "Approach the following question the way someone with a PhD in biology would.json",
               "Approach the following question the way someone with a PhD in literature would.json",
               "Approach the following question the way someone with a PhD in philosophy would.json"
              ]

for dir_name in os.listdir(directory_path):
    dir_path = os.path.join(directory_path, dir_name)

    if os.path.isdir(dir_path):
        directory_data = {'Directory Name': int(dir_name)}

        for json_file in json_files:
            json_file_path = os.path.join(dir_path, json_file)
            try:
                with open(json_file_path, 'r') as f:
                    json_data = json.load(f)
                    directory_data[json_file] = json_data['yes_no']
            except FileNotFoundError:
                directory_data[json_file] = 'N/A'

        data.append(directory_data)

sorted_data = sorted(data, key=lambda x: x['Directory Name'])

with open(csv_file_path, 'w', newline='') as csvfile:
    fieldnames = ['Directory Name'] + json_files
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for row in sorted_data:
        writer.writerow(row)

print("Done:", csv_file_path)

### Analyze CSV

In [16]:
csv_file_paths = ["/content/drive/My Drive/Research/Multi Agent/MMLU_college_mathematics_domain_experts_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/MMLU_abstract_algebra_domain_experts_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/MMLU_college_physics_domain_experts_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/MMLU_philosophy_domain_experts_data.csv"
                  ]

for csv_file_path in csv_file_paths:
    print(csv_file_path)

    unique_combinations = {}

    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)

        for row in reader:
            row_values = row[1:]

            combination = ''.join([val[:1] for val in row_values])

            unique_combinations[combination] = unique_combinations.get(combination, 0) + 1

    table_data = []
    total_count = 0
    for combination, count in unique_combinations.items():
        mathematics = combination[0]
        physics = combination[1]
        engineering = combination[2]
        computer_science = combination[3]
        biology = combination[4]
        literature = combination[5]
        philosophy = combination[6]
        table_data.append([mathematics, physics, engineering, computer_science, biology, literature, philosophy, count])
        total_count += count

    table_data.append(["Total", "", "", "", "", "", "", total_count])

    print(tabulate(table_data, headers=['mathematics', 'physics', 'engineering', 'computer_science', 'biology', 'literature', 'philosophy', 'Count']))
    print("-----------------------------------------------------------------------------------------------------------\n")

/content/drive/My Drive/Research/Multi Agent/MMLU_college_mathematics_domain_experts_data.csv
mathematics    physics    engineering    computer_science    biology    literature    philosophy      Count
-------------  ---------  -------------  ------------------  ---------  ------------  ------------  -------
y              y          y              y                   y          y             y                  59
y              y          y              y                   y          n             y                   7
n              y          y              n                   y          y             y                   1
y              y          y              y                   y          y             n                   4
y              y          y              y                   n          y             n                   1
y              y          n              y                   y          y             y                   2
y              y          y              n

In [32]:
def domain_specific_analyzing(csv_file_path, subject, index):
    print(csv_file_path)

    unique_combinations = {
        'All Yes': 0,
        'All No': 0,
        f'{subject} Yes': 0,
        f'{subject} No': 0,
        'Error or N/A': 0
    }

    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)

        for row in reader:
            row_values = row[1:]

            all_yes = all(val.lower() == 'yes' for val in row_values)
            all_no = all(val.lower() == 'no' for val in row_values)
            subject_yes = row_values[index].lower() == 'yes'
            subject_no = row_values[index].lower() == 'no'

            if all_yes:
                unique_combinations['All Yes'] += 1
            elif all_no:
                unique_combinations['All No'] += 1
            elif subject_yes:
                unique_combinations[f'{subject} Yes'] += 1
            elif subject_no:
                unique_combinations[f'{subject} No'] += 1
            else:
                unique_combinations['Error or N/A'] += 1

    table_data = []

    for combination, count in unique_combinations.items():
        table_data.append([combination, count])

    print(tabulate(table_data, headers=['Combination', 'Count']))
    print("-----------------------------------------------------------------------------------------------------------\n")

In [33]:
csv_file_paths = ["/content/drive/My Drive/Research/Multi Agent/MMLU_college_mathematics_domain_experts_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/MMLU_abstract_algebra_domain_experts_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/MMLU_college_physics_domain_experts_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/MMLU_philosophy_domain_experts_data.csv"
                  ]

subjects = ['mathematics',
            'mathematics',
            'physics',
            'philosophy'
            ]

indices = [0,
           0,
           1,
           6
           ]

for i in range(len(csv_file_paths)):
    domain_specific_analyzing(csv_file_paths[i], subjects[i], i)

/content/drive/My Drive/Research/Multi Agent/MMLU_college_mathematics_domain_experts_data.csv
Combination        Count
---------------  -------
All Yes               59
All No                 0
mathematics Yes       34
mathematics No         7
Error or N/A           0
-----------------------------------------------------------------------------------------------------------

/content/drive/My Drive/Research/Multi Agent/MMLU_abstract_algebra_domain_experts_data.csv
Combination        Count
---------------  -------
All Yes               44
All No                 1
mathematics Yes       33
mathematics No        21
Error or N/A           1
-----------------------------------------------------------------------------------------------------------

/content/drive/My Drive/Research/Multi Agent/MMLU_college_physics_domain_experts_data.csv
Combination      Count
-------------  -------
All Yes             53
All No               0
physics Yes         32
physics No          17
Error or N/A       

# SciEval

## Prompt Prefixes

In [9]:
impersonation = ["Approach the following question the way someone with a PhD in mathematics would. ",
                 "Approach the following question the way someone with a PhD in physics would. ",
                 "Approach the following question the way someone with a PhD in engineering would. ",
                 "Approach the following question the way someone with a PhD in computer science would. ",
                 "Approach the following question the way someone with a PhD in biology would. ",
                 "Approach the following question the way someone with a PhD in literature would. ",
                 "Approach the following question the way someone with a PhD in philosophy would. "
                 ]

## Processing

In [10]:
def process_json(file_path, output_directory):
    with open(file_path, 'r') as f:
        data = json.load(f)

    row_num = 0
    for entryNum in range(len(data)):
        row_num += 1

        question_dir = os.path.join(output_directory, str(row_num))
        if not os.path.exists(question_dir):
            os.makedirs(question_dir, exist_ok=True)

        json_files_count = len([filename for filename in os.listdir(question_dir) if filename.endswith(".json")])
        if json_files_count == len(impersonation):
            print(f"Skipping {question_dir} (already complete)")
            continue

        question = data[entryNum]['question']
        solution = data[entryNum]['answer']

        problem_prompt = question

        for prefix in impersonation:
            problem_prompt_with_prefix = prefix + problem_prompt

            difficulty_prompt = "On a scale of 1-10, how difficult is this problem? Respond with just the number.: " + problem_prompt
            gpt_response_difficulty = chat_gpt(difficulty_prompt)
            difficulty_value = get_difficulty(gpt_response_difficulty)

            for i in range(num_attempts):
                attempts = i + 1
                gpt_response = chat_gpt(problem_prompt_with_prefix)
                solution_value = solution
                solution_prompt = "The following is a response to a question. Yes or no, is the final solution obtained in the following response " + str(solution_value) + "?: " + gpt_response.choices[0].message.content.strip()
                gpt_response_correct_incorrect = chat_gpt(solution_prompt)
                yes_no = get_yes_no(gpt_response_correct_incorrect)
                if yes_no == 'yes':
                    break;

            name = prefix[:prefix.rfind(".")]
            output_filename = f"{name}.json"
            output_file_path = os.path.join(question_dir, output_filename)

            with open(output_file_path, 'w') as output_file:
                json.dump({
                    'prefix': prefix,
                    'problem_prompt': problem_prompt_with_prefix,
                    'solution': solution,
                    'gpt_response': str(gpt_response.choices[0].message.content.strip()),
                    'solution_prompt': solution_prompt,
                    'gpt_response_correct_incorrect': str(gpt_response_correct_incorrect.choices[0].message.content.strip()),
                    'attempts': attempts,
                    'yes_no': yes_no,
                    'gpt_response_difficulty':str(gpt_response_difficulty.choices[0].message.content.strip()),
                    'difficulty_value': difficulty_value
                }, output_file, indent=4)

        print(f'{row_num}/{len(data)}')

In [11]:
num_attempts = 3

file_path = "/content/drive/My Drive/Research/Multi Agent/Datasets/SciEval/SciEval-main/dynamic_chem.json"
output_directory = "/content/drive/My Drive/Research/Multi Agent/SciEval/chemistry"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

process_json(file_path, output_directory)

print("Done")

Skipping /content/drive/My Drive/Research/Multi Agent/SciEval/chemistry/1 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/SciEval/chemistry/2 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/SciEval/chemistry/3 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/SciEval/chemistry/4 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/SciEval/chemistry/5 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/SciEval/chemistry/6 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/SciEval/chemistry/7 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/SciEval/chemistry/8 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/SciEval/chemistry/9 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/SciEval/chemistry/10 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/SciEval/chemist

KeyboardInterrupt: 

## Interpreting Results with CSV

### Generate CSV

In [17]:
directory_path = "/content/drive/My Drive/Research/Multi Agent/SciEval/chemistry"
csv_file_path = "/content/drive/My Drive/Research/Multi Agent/SciEval_chemistry_data.csv"

data = []
json_files = ["Approach the following question the way someone with a PhD in mathematics would.json",
               "Approach the following question the way someone with a PhD in physics would.json",
               "Approach the following question the way someone with a PhD in engineering would.json",
               "Approach the following question the way someone with a PhD in computer science would.json",
               "Approach the following question the way someone with a PhD in biology would.json",
               "Approach the following question the way someone with a PhD in literature would.json",
               "Approach the following question the way someone with a PhD in philosophy would.json"
              ]

for dir_name in os.listdir(directory_path):
    dir_path = os.path.join(directory_path, dir_name)

    if os.path.isdir(dir_path):
        directory_data = {'Directory Name': int(dir_name)}

        for json_file in json_files:
            json_file_path = os.path.join(dir_path, json_file)
            try:
                with open(json_file_path, 'r') as f:
                    json_data = json.load(f)
                    directory_data[json_file] = json_data['yes_no']
            except FileNotFoundError:
                directory_data[json_file] = 'N/A'

        data.append(directory_data)

sorted_data = sorted(data, key=lambda x: x['Directory Name'])

with open(csv_file_path, 'w', newline='') as csvfile:
    fieldnames = ['Directory Name'] + json_files
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for row in sorted_data:
        writer.writerow(row)

print("Done:", csv_file_path)

Done: /content/drive/My Drive/Research/Multi Agent/SciEval_chemistry_data.csv


### Analyze CSV

In [18]:
csv_file_paths = ["/content/drive/My Drive/Research/Multi Agent/SciEval_physics_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/SciEval_chemistry_data.csv"
                  ]

for csv_file_path in csv_file_paths:
    print(csv_file_path)

    unique_combinations = {}

    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)

        for row in reader:
            row_values = row[1:]

            combination = ''.join([val[:1] for val in row_values])

            unique_combinations[combination] = unique_combinations.get(combination, 0) + 1

    table_data = []
    total_count = 0
    for combination, count in unique_combinations.items():
        mathematics = combination[0]
        physics = combination[1]
        engineering = combination[2]
        computer_science = combination[3]
        biology = combination[4]
        literature = combination[5]
        philosophy = combination[6]
        table_data.append([mathematics, physics, engineering, computer_science, biology, literature, philosophy, count])
        total_count += count

    table_data.append(["Total", "", "", "", "", "", "", total_count])

    print(tabulate(table_data, headers=['mathematics', 'physics', 'engineering', 'computer_science', 'biology', 'literature', 'philosophy', 'Count']))
    print("-----------------------------------------------------------------------------------------------------------\n")

/content/drive/My Drive/Research/Multi Agent/SciEval_physics_data.csv
mathematics    physics    engineering    computer_science    biology    literature    philosophy      Count
-------------  ---------  -------------  ------------------  ---------  ------------  ------------  -------
n              n          y              y                   y          y             n                   1
n              n          y              y                   y          y             y                   2
y              y          n              n                   n          y             n                   2
y              y          y              n                   y          y             y                   3
y              y          y              y                   y          y             y                  16
y              n          n              y                   y          y             y                   2
y              y          y              n                   n    

In [19]:
def domain_specific_analyzing(csv_file_path, subject, index):
    print(csv_file_path)

    unique_combinations = {
        'All Yes': 0,
        'All No': 0,
        f'{subject} Yes': 0,
        f'{subject} No': 0,
        'Error or N/A': 0
    }

    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)

        for row in reader:
            row_values = row[1:]

            all_yes = all(val.lower() == 'yes' for val in row_values)
            all_no = all(val.lower() == 'no' for val in row_values)
            subject_yes = row_values[index].lower() == 'yes'
            subject_no = row_values[index].lower() == 'no'

            if all_yes:
                unique_combinations['All Yes'] += 1
            elif all_no:
                unique_combinations['All No'] += 1
            elif subject_yes:
                unique_combinations[f'{subject} Yes'] += 1
            elif subject_no:
                unique_combinations[f'{subject} No'] += 1
            else:
                unique_combinations['Error or N/A'] += 1

    table_data = []

    for combination, count in unique_combinations.items():
        table_data.append([combination, count])

    print(tabulate(table_data, headers=['Combination', 'Count']))
    print("-----------------------------------------------------------------------------------------------------------\n")

In [24]:
csv_file_paths = ["/content/drive/My Drive/Research/Multi Agent/SciEval_physics_data.csv"
                  ]

subjects = ['physics'
            ]

indices = [1
           ]

for i in range(len(csv_file_paths)):
    domain_specific_analyzing(csv_file_paths[i], subjects[i], indices[i])

/content/drive/My Drive/Research/Multi Agent/SciEval_physics_data.csv
Combination      Count
-------------  -------
All Yes             16
All No              35
physics Yes         77
physics No         123
Error or N/A         0
-----------------------------------------------------------------------------------------------------------



# AGIEval

## Prompt Prefixes

In [43]:
impersonation = ["Approach the following question the way someone with a PhD in mathematics would. ",
                 "Approach the following question the way someone with a PhD in physics would. ",
                 "Approach the following question the way someone with a PhD in engineering would. ",
                 "Approach the following question the way someone with a PhD in computer science would. ",
                 "Approach the following question the way someone with a PhD in biology would. ",
                 "Approach the following question the way someone with a PhD in literature would. ",
                 "Approach the following question the way someone with a PhD in philosophy would. "
                 ]

## Processing

In [61]:
def process_json(file_path, output_directory):
    with jsonlines.open(file_path) as f:
        data = [line for line in f]

    row_num = 0
    for entryNum in range(len(data)):
        row_num += 1

        question_dir = os.path.join(output_directory, str(row_num))
        if not os.path.exists(question_dir):
            os.makedirs(question_dir, exist_ok=True)

        json_files_count = len([filename for filename in os.listdir(question_dir) if filename.endswith(".json")])
        if json_files_count == len(impersonation):
            print(f"Skipping {question_dir} (already complete)")
            continue

        question = data[entryNum]['question']
        solution = data[entryNum]['answer']

        problem_prompt = question

        for prefix in impersonation:
            problem_prompt_with_prefix = prefix + problem_prompt

            difficulty_prompt = "On a scale of 1-10, how difficult is this problem? Respond with just the number.: " + problem_prompt
            gpt_response_difficulty = chat_gpt(difficulty_prompt)
            difficulty_value = get_difficulty(gpt_response_difficulty)

            for i in range(num_attempts):
                attempts = i + 1
                gpt_response = chat_gpt(problem_prompt_with_prefix)
                solution_value = solution
                solution_prompt = "The following is a response to a question. Yes or no, is the final solution obtained in the following response " + str(solution_value) + "?: " + gpt_response.choices[0].message.content.strip()
                gpt_response_correct_incorrect = chat_gpt(solution_prompt)
                yes_no = get_yes_no(gpt_response_correct_incorrect)
                if yes_no == 'yes':
                    break;

            name = prefix[:prefix.rfind(".")]
            output_filename = f"{name}.json"
            output_file_path = os.path.join(question_dir, output_filename)

            with open(output_file_path, 'w') as output_file:
                json.dump({
                    'prefix': prefix,
                    'problem_prompt': problem_prompt_with_prefix,
                    'solution': solution,
                    'gpt_response': str(gpt_response.choices[0].message.content.strip()),
                    'solution_prompt': solution_prompt,
                    'gpt_response_correct_incorrect': str(gpt_response_correct_incorrect.choices[0].message.content.strip()),
                    'attempts': attempts,
                    'yes_no': yes_no,
                    'gpt_response_difficulty':str(gpt_response_difficulty.choices[0].message.content.strip()),
                    'difficulty_value': difficulty_value
                }, output_file, indent=4)

        print(f'{row_num}/{len(data)}')

In [68]:
num_attempts = 3

file_path = "/content/drive/My Drive/Research/Multi Agent/Datasets/AGIEval/AGIEval-main/data/v1/math.jsonl"
output_directory = "/content/drive/My Drive/Research/Multi Agent/AGIEval/math"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

process_json(file_path, output_directory)

print("Done")

Skipping /content/drive/My Drive/Research/Multi Agent/AGIEval/math/1 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/AGIEval/math/2 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/AGIEval/math/3 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/AGIEval/math/4 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/AGIEval/math/5 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/AGIEval/math/6 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/AGIEval/math/7 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/AGIEval/math/8 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/AGIEval/math/9 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/AGIEval/math/10 (already complete)
Skipping /content/drive/My Drive/Research/Multi Agent/AGIEval/math/11 (already complete)
Skipping /content/drive/My Dri

KeyboardInterrupt: 

## Interpreting Results with CSV

### Generate CSV

In [69]:
directory_path = "/content/drive/My Drive/Research/Multi Agent/AGIEval/math"
csv_file_path = "/content/drive/My Drive/Research/Multi Agent/AGIEval_math_data.csv"

data = []
json_files = ["Approach the following question the way someone with a PhD in mathematics would.json",
               "Approach the following question the way someone with a PhD in physics would.json",
               "Approach the following question the way someone with a PhD in engineering would.json",
               "Approach the following question the way someone with a PhD in computer science would.json",
               "Approach the following question the way someone with a PhD in biology would.json",
               "Approach the following question the way someone with a PhD in literature would.json",
               "Approach the following question the way someone with a PhD in philosophy would.json"
              ]

for dir_name in os.listdir(directory_path):
    dir_path = os.path.join(directory_path, dir_name)

    if os.path.isdir(dir_path):
        directory_data = {'Directory Name': int(dir_name)}

        for json_file in json_files:
            json_file_path = os.path.join(dir_path, json_file)
            try:
                with open(json_file_path, 'r') as f:
                    json_data = json.load(f)
                    directory_data[json_file] = json_data['yes_no']
            except FileNotFoundError:
                directory_data[json_file] = 'N/A'

        data.append(directory_data)

sorted_data = sorted(data, key=lambda x: x['Directory Name'])

with open(csv_file_path, 'w', newline='') as csvfile:
    fieldnames = ['Directory Name'] + json_files
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for row in sorted_data:
        writer.writerow(row)

print("Done:", csv_file_path)

Done: /content/drive/My Drive/Research/Multi Agent/AGIEval_math_data.csv


### Analyze CSV

In [70]:
csv_file_paths = ["/content/drive/My Drive/Research/Multi Agent/AGIEval_math_data.csv"
                  ]

for csv_file_path in csv_file_paths:
    print(csv_file_path)

    unique_combinations = {}

    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)

        for row in reader:
            row_values = row[1:]

            combination = ''.join([val[:1] for val in row_values])

            unique_combinations[combination] = unique_combinations.get(combination, 0) + 1

    table_data = []
    total_count = 0
    for combination, count in unique_combinations.items():
        mathematics = combination[0]
        physics = combination[1]
        engineering = combination[2]
        computer_science = combination[3]
        biology = combination[4]
        literature = combination[5]
        philosophy = combination[6]
        table_data.append([mathematics, physics, engineering, computer_science, biology, literature, philosophy, count])
        total_count += count

    table_data.append(["Total", "", "", "", "", "", "", total_count])

    print(tabulate(table_data, headers=['mathematics', 'physics', 'engineering', 'computer_science', 'biology', 'literature', 'philosophy', 'Count']))
    print("-----------------------------------------------------------------------------------------------------------\n")

/content/drive/My Drive/Research/Multi Agent/AGIEval_math_data.csv
mathematics    physics    engineering    computer_science    biology    literature    philosophy      Count
-------------  ---------  -------------  ------------------  ---------  ------------  ------------  -------
y              y          y              y                   y          y             n                  10
y              y          y              y                   y          y             y                  86
n              n          y              n                   y          n             n                   3
y              n          y              n                   y          y             n                   1
n              n          y              n                   n          n             y                   3
y              n          n              n                   y          n             n                   2
y              y          y              n                   n       

In [71]:
def domain_specific_analyzing(csv_file_path, subject, index):
    print(csv_file_path)

    unique_combinations = {
        'All Yes': 0,
        'All No': 0,
        f'{subject} Yes': 0,
        f'{subject} No': 0,
        'Error or N/A': 0
    }

    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)

        for row in reader:
            row_values = row[1:]

            all_yes = all(val.lower() == 'yes' for val in row_values)
            all_no = all(val.lower() == 'no' for val in row_values)
            subject_yes = row_values[index].lower() == 'yes'
            subject_no = row_values[index].lower() == 'no'

            if all_yes:
                unique_combinations['All Yes'] += 1
            elif all_no:
                unique_combinations['All No'] += 1
            elif subject_yes:
                unique_combinations[f'{subject} Yes'] += 1
            elif subject_no:
                unique_combinations[f'{subject} No'] += 1
            else:
                unique_combinations['Error or N/A'] += 1

    table_data = []

    for combination, count in unique_combinations.items():
        table_data.append([combination, count])

    print(tabulate(table_data, headers=['Combination', 'Count']))
    print("-----------------------------------------------------------------------------------------------------------\n")

In [72]:
csv_file_paths = ["/content/drive/My Drive/Research/Multi Agent/AGIEval_math_data.csv"
                  ]

subjects = ['mathematics'
            ]

indices = [0,
           ]

for i in range(len(csv_file_paths)):
    domain_specific_analyzing(csv_file_paths[i], subjects[i], indices[i])

/content/drive/My Drive/Research/Multi Agent/AGIEval_math_data.csv
Combination        Count
---------------  -------
All Yes               86
All No                14
mathematics Yes       94
mathematics No        77
Error or N/A           0
-----------------------------------------------------------------------------------------------------------



# MultiMedQA

## Prompt Prefixes

In [12]:
impersonation = ["Approach the following multiple choice question the way a physician would. ",
                 "Approach the following multiple choice question the way a medical researcher would. ",
                 "Approach the following multiple choice question the way a doctor would. ",
                 "Solve the following multiple choice question. "
                 ]

## Processing

In [13]:
def process_json(file_path, output_directory):
    with jsonlines.open(file_path) as f:
        data = [line for line in f]

    row_num = 0
    for entryNum in range(len(data)):
        row_num += 1

        question_dir = os.path.join(output_directory, str(row_num))
        if not os.path.exists(question_dir):
            os.makedirs(question_dir, exist_ok=True)

        json_files_count = len([filename for filename in os.listdir(question_dir) if filename.endswith(".json")])
        if json_files_count == len(impersonation):
            print(f"Skipping {question_dir} (already complete)")
            continue

        if data[entryNum]['choice_type'] == 'multi':
            print(f"Skipping {question_dir} (multi select)")
            continue

        question = data[entryNum]['question']
        solution = data[entryNum]['cop']

        options = "\n1: " + data[entryNum]['opa'] + "\n2: " + data[entryNum]['opb'] + "\n3: " + data[entryNum]['opc'] + "\n4: " + data[entryNum]['opd']
        problem_prompt = question + options

        for prefix in impersonation:
            problem_prompt_with_prefix = prefix + problem_prompt

            difficulty_prompt = "On a scale of 1-10, how difficult is this problem? Respond with just the number.: " + problem_prompt
            gpt_response_difficulty = chat_gpt(difficulty_prompt)
            difficulty_value = get_difficulty(gpt_response_difficulty)

            for i in range(num_attempts):
                attempts = i + 1
                gpt_response = chat_gpt(problem_prompt_with_prefix)
                solution_value = solution
                solution_prompt = "The following is a response to a question, with options 1, 2, 3, 4. Yes or no, is the final solution obtained in the following response " + str(solution_value) + "?: " + gpt_response.choices[0].message.content.strip()
                gpt_response_correct_incorrect = chat_gpt(solution_prompt)
                yes_no = get_yes_no(gpt_response_correct_incorrect)
                if yes_no == 'yes':
                    break;

            name = prefix[:prefix.rfind(".")]
            output_filename = f"{name}.json"
            output_file_path = os.path.join(question_dir, output_filename)

            with open(output_file_path, 'w') as output_file:
                json.dump({
                    'prefix': prefix,
                    'problem_prompt': problem_prompt_with_prefix,
                    'solution': solution,
                    'gpt_response': str(gpt_response.choices[0].message.content.strip()),
                    'solution_prompt': solution_prompt,
                    'gpt_response_correct_incorrect': str(gpt_response_correct_incorrect.choices[0].message.content.strip()),
                    'attempts': attempts,
                    'yes_no': yes_no,
                    'gpt_response_difficulty':str(gpt_response_difficulty.choices[0].message.content.strip()),
                    'difficulty_value': difficulty_value
                }, output_file, indent=4)

        print(f'{row_num}/{len(data)}')

In [14]:
num_attempts = 3

file_path = "/content/drive/My Drive/Research/Multi Agent/Datasets/medmcqa/train.json"
output_directory = "/content/drive/My Drive/Research/Multi Agent/medmcqa/train"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

process_json(file_path, output_directory)

print("Done")

1/182822
2/182822
Skipping /content/drive/My Drive/Research/Multi Agent/medmcqa/train/3 (multi select)
Skipping /content/drive/My Drive/Research/Multi Agent/medmcqa/train/4 (multi select)
5/182822
6/182822
Skipping /content/drive/My Drive/Research/Multi Agent/medmcqa/train/7 (multi select)
8/182822
9/182822
10/182822
11/182822
Skipping /content/drive/My Drive/Research/Multi Agent/medmcqa/train/12 (multi select)
13/182822
Skipping /content/drive/My Drive/Research/Multi Agent/medmcqa/train/14 (multi select)
15/182822
16/182822
17/182822
Skipping /content/drive/My Drive/Research/Multi Agent/medmcqa/train/18 (multi select)
19/182822
20/182822
Skipping /content/drive/My Drive/Research/Multi Agent/medmcqa/train/21 (multi select)
22/182822
Skipping /content/drive/My Drive/Research/Multi Agent/medmcqa/train/23 (multi select)
24/182822
25/182822
26/182822
27/182822
28/182822
29/182822
30/182822
Skipping /content/drive/My Drive/Research/Multi Agent/medmcqa/train/31 (multi select)
Skipping /conte

KeyboardInterrupt: 

## Interpreting Results with CSV

### Generate CSV

In [20]:
directory_path = "/content/drive/My Drive/Research/Multi Agent/medmcqa/train"
csv_file_path = "/content/drive/My Drive/Research/Multi Agent/medmcqa_train_data.csv"

data = []
json_files = ["Approach the following multiple choice question the way a physician would.json",
                 "Approach the following multiple choice question the way a medical researcher would.json",
                 "Approach the following multiple choice question the way a doctor would.json",
                 "Solve the following multiple choice question.json"
                 ]

for dir_name in os.listdir(directory_path):
    dir_path = os.path.join(directory_path, dir_name)

    if os.path.isdir(dir_path):
        directory_data = {'Directory Name': int(dir_name)}

        for json_file in json_files:
            json_file_path = os.path.join(dir_path, json_file)
            try:
                with open(json_file_path, 'r') as f:
                    json_data = json.load(f)
                    directory_data[json_file] = json_data['yes_no']
            except FileNotFoundError:
                directory_data[json_file] = 'N/A'

        data.append(directory_data)

sorted_data = sorted(data, key=lambda x: x['Directory Name'])

with open(csv_file_path, 'w', newline='') as csvfile:
    fieldnames = ['Directory Name'] + json_files
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for row in sorted_data:
        writer.writerow(row)

print("Done:", csv_file_path)

Done: /content/drive/My Drive/Research/Multi Agent/medmcqa_train_data.csv


### Analyze CSV

In [21]:
csv_file_paths = ["/content/drive/My Drive/Research/Multi Agent/medmcqa_train_data.csv"
                  ]

for csv_file_path in csv_file_paths:
    print(csv_file_path)

    unique_combinations = {}

    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)

        for row in reader:
            row_values = row[1:]

            combination = ''.join([val[:1] for val in row_values])

            unique_combinations[combination] = unique_combinations.get(combination, 0) + 1

    table_data = []
    total_count = 0
    for combination, count in unique_combinations.items():
        physician = combination[0]
        medical_researcher = combination[1]
        doctor = combination[2]
        regular = combination[3]
        table_data.append([physician, medical_researcher, doctor, regular, count])
        total_count += count

    table_data.append(["Total", "", "", "", "", total_count])

    print(tabulate(table_data, headers=['physician', 'medical_researcher', 'doctor', 'regular', 'Count']))
    print("-----------------------------------------------------------------------------------------------------------\n")

/content/drive/My Drive/Research/Multi Agent/medmcqa_train_data.csv
physician    medical_researcher    doctor    regular    Count
-----------  --------------------  --------  ---------  -------
y            y                     y         y          135
n            y                     n         n          29
N            N                     N         N          237
n            n                     y         n          9
y            y                     y         n          109
n            y                     y         y          22
n            y                     y         n          33
y            n                     n         y          6
n            y                     n         y          11
y            n                     n         n          14
y            y                     n         n          29
n            n                     n         y          5
n            n                     n         n          24
y            n                     y   

# Partitioning by Sentence (Math)

## Processing

In [9]:
parent_directory = "/content/drive/My Drive/Research/Meeting Prep/6Dataset 1/MATHByLevel/train/number_theory"
output_base_directory = "/content/drive/My Drive/Research/Multi Agent/MATHByLevel_partitioning/train/number_theory"

pattern = r'\\boxed{([^}]*)}'
os.makedirs(output_base_directory, exist_ok=True)

In [55]:
def process_file(input_file_path, output_directory):

    filename = os.path.basename(input_file_path)
    output_directory_name = os.path.splitext(filename)[0]
    output_directory = os.path.join(output_directory, output_directory_name)

    with open(input_file_path, 'r') as file:
        data = json.load(file)

        problem_prompt = data.get('problem')
        problem_sentences = sent_tokenize(problem_prompt)
        solution = data.get('solution')

        if len(problem_sentences) == 1:
            print(f"Skipping {output_directory} (single sentence)")
            return

        os.makedirs(output_directory, exist_ok=True)

        json_files_count = len([filename for filename in os.listdir(output_directory) if filename.endswith(".json")])
        if json_files_count == len(problem_sentences):
            print(f"Skipping {output_directory} (already complete)")
            return

        initial_prompt = "Your job is to consider a number theory problem, and leave a plan for the next person to solve it. The problem will be revealed one sentence at a time. You do not have the entire problem yet, thus just do whatever calcluations/planning you think will be useful for the next person. If you don't think you can make any progress, say so. The following is a portion of the entire problem, and the work/progress/message the previous person left for you. There may be errors in the provided work that was previously done and left for you, so check and fix them if necessary."
        current_solving_state = ""
        current_problem_state = ""

        for index, sentence in enumerate(problem_sentences):
            current_problem_state += sentence

            if index == len(problem_sentences) - 1:
                initial_prompt = "Your job is to solve a number theory problem. The problem was revealed one sentence at a time. You are on the last sentence have the entire problem now. The following is as much of the problem you have revealed, and the work done so far. There may be errors in the work done so far, so fix them if necessary."
            problem_prompt = initial_prompt + "\n" + current_problem_state + "\n" + current_solving_state

            match = re.search(pattern, solution)

            if match:
                gpt_response = chat_gpt(problem_prompt).choices[0].message.content.strip()
                # current_solving_state += "\n" + gpt_response
                current_solving_state = gpt_response

                if index == len(problem_sentences) - 1:
                    solution_value = match.group(1)
                    solution_prompt = "The following is a response to a question. Yes or no, is the final solution obtained in the following response " + str(solution_value) + "?: " + gpt_response
                    gpt_response_correct_incorrect = chat_gpt(solution_prompt).choices[0].message.content.strip().lower()
                    yes_no = get_yes_no(gpt_response_correct_incorrect)
                else:
                    solution_prompt = "Question partition not complete yet"
                    gpt_response_correct_incorrect = "Question partition not complete yet"
                    yes_no = "Question partition not complete yet"

                name = str(index)

                output_filename = f"{name}.json"
                output_file_path = os.path.join(output_directory, output_filename)
                with open(output_file_path, 'w') as output_file:
                    json.dump({
                        'problem_prompt': problem_prompt,
                        'solution': solution,
                        'gpt_response': gpt_response,
                        'solution_prompt': solution_prompt,
                        'gpt_response_correct_incorrect': gpt_response_correct_incorrect,
                        'yes_no': yes_no
                    }, output_file, indent=4)

            else:
                print("No text inside \\boxed{} found in the solution string:", output_directory)

def process_directory(input_directory, output_base_directory):
    output_directory = os.path.join(output_base_directory, os.path.basename(input_directory))
    os.makedirs(output_directory, exist_ok=True)

    counter = 1
    for filename in os.listdir(input_directory):
        if filename.endswith(".json"):
            input_file_path = os.path.join(input_directory, filename)
            process_file(input_file_path, output_directory)
            print(f'{counter}/{len(os.listdir(input_directory))}')
            counter += 1

In [58]:
num_attempts = 1
level = 1

for directory in os.listdir(parent_directory):
    input_directory = os.path.join(parent_directory, directory)

    print("Level:", level)
    if os.path.isdir(input_directory):
        process_directory(input_directory, output_base_directory)

    level += 1

print("Done")

Level: 1
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_partitioning/train/number_theory/Level_1/507 (single sentence)
1/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_partitioning/train/number_theory/Level_1/1054 (single sentence)
2/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_partitioning/train/number_theory/Level_1/446 (single sentence)
3/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_partitioning/train/number_theory/Level_1/15 (already complete)
4/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_partitioning/train/number_theory/Level_1/347 (single sentence)
5/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_partitioning/train/number_theory/Level_1/49 (single sentence)
6/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_partitioning/train/number_theory/Level_1/722 (already complete)
7/47
Skipping /content/drive/My Drive/Research/Multi Agen

## Interpreting Results

### Proportion correct

In [24]:
directory_paths = ["/content/drive/My Drive/Research/Multi Agent/MATHByLevel_partitioning/train/number_theory/Level_1",
                  "/content/drive/My Drive/Research/Multi Agent/MATHByLevel_partitioning/train/number_theory/Level_2",
                  "/content/drive/My Drive/Research/Multi Agent/MATHByLevel_partitioning/train/number_theory/Level_3",
                  "/content/drive/My Drive/Research/Multi Agent/MATHByLevel_partitioning/train/number_theory/Level_4",
                  "/content/drive/My Drive/Research/Multi Agent/MATHByLevel_partitioning/train/number_theory/Level_5"
                  ]

count = 0
total = 0

for index, directory_path in enumerate(directory_paths, start=1):

    for dir_name in os.listdir(directory_path):
        dir_path = os.path.join(directory_path, dir_name)

        if os.path.isdir(dir_path):
            directory_data = {'Directory Name': int(dir_name)}

            json_files = [file for file in (os.listdir(dir_path)) if file.endswith('.json')]
            if len(json_files) == 0:
                continue

            json_file = sorted(json_files)[-1]
            json_file_path = os.path.join(dir_path, json_file)

            try:
                with open(json_file_path, 'r') as f:
                    json_data = json.load(f)
                    if json_data['yes_no'] == 'yes':
                        count += 1
                    elif json_data['yes_no'] == 'no':
                        count += 0
                    total += 1
            except FileNotFoundError:
                print("File Not Found")

    print(f'Level {index}: {count}/{total}')

print("\nDone")

Level 1: 10/22
Level 2: 39/76
Level 3: 73/142
Level 4: 103/227
Level 5: 137/415

Done


# Base Cases (MATH)

## Prompt Prefixes

In [10]:
impersonation = ["Solve the following problem. "
                 ]

## Processing

In [11]:
parent_directory = "/content/drive/My Drive/Research/Meeting Prep/6Dataset 1/MATHByLevel/train/number_theory"
output_base_directory = "/content/drive/My Drive/Research/Multi Agent/MATHByLevel_base/train/number_theory"

pattern = r'\\boxed{([^}]*)}'
os.makedirs(output_base_directory, exist_ok=True)

In [12]:
def process_file(input_file_path, output_directory):

    filename = os.path.basename(input_file_path)
    output_directory_name = os.path.splitext(filename)[0]
    output_directory = os.path.join(output_directory, output_directory_name)
    os.makedirs(output_directory, exist_ok=True)

    json_files_count = len([filename for filename in os.listdir(output_directory) if filename.endswith(".json")])
    if json_files_count == len(impersonation):
        print(f"Skipping {output_directory} (already complete)")
        return

    with open(input_file_path, 'r') as file:
        data = json.load(file)

        problem_prompt = data.get('problem')
        solution = data.get('solution')

        for prefix in impersonation:
            problem_prompt_with_prefix = prefix + problem_prompt

            match = re.search(pattern, solution)

            if match:
                for i in range(num_attempts):
                    attempts = i + 1
                    gpt_response = chat_gpt(problem_prompt_with_prefix)
                    solution_value = match.group(1)
                    solution_prompt = "The following is a response to a question. Yes or no, is the final solution obtained in the following response " + str(solution_value) + "?: " + gpt_response.choices[0].message.content.strip()
                    gpt_response_correct_incorrect = chat_gpt(solution_prompt)
                    yes_no = get_yes_no(gpt_response_correct_incorrect)
                    if yes_no == 'yes':
                        break;

                name = prefix[:prefix.rfind(".")]
                output_filename = f"{name}.json"
                output_file_path = os.path.join(output_directory, output_filename)
                with open(output_file_path, 'w') as output_file:
                    json.dump({
                        'prefix': prefix,
                        'problem_prompt': problem_prompt_with_prefix,
                        'solution': solution,
                        'gpt_response': str(gpt_response.choices[0].message.content.strip()),
                        'solution_prompt': solution_prompt,
                        'gpt_response_correct_incorrect': str(gpt_response_correct_incorrect.choices[0].message.content.strip()),
                        'attempts': attempts,
                        'yes_no': yes_no
                    }, output_file, indent=4)

            else:
                print("No text inside \\boxed{} found in the solution string:", output_directory)

def process_directory(input_directory, output_base_directory):
    output_directory = os.path.join(output_base_directory, os.path.basename(input_directory))
    os.makedirs(output_directory, exist_ok=True)

    counter = 1
    for filename in os.listdir(input_directory):
        if filename.endswith(".json"):
            input_file_path = os.path.join(input_directory, filename)
            process_file(input_file_path, output_directory)
            print(f'{counter}/{len(os.listdir(input_directory))}')
            counter += 1

In [15]:
num_attempts = 3
level = 1

for directory in os.listdir(parent_directory):
    input_directory = os.path.join(parent_directory, directory)

    print("Level:", level)
    if os.path.isdir(input_directory):
        process_directory(input_directory, output_base_directory)

    level += 1

print("Done")

Level: 1
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_base/train/number_theory/Level_1/507 (already complete)
1/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_base/train/number_theory/Level_1/1054 (already complete)
2/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_base/train/number_theory/Level_1/446 (already complete)
3/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_base/train/number_theory/Level_1/15 (already complete)
4/47
5/47
6/47
7/47
8/47
9/47
10/47
11/47
12/47
13/47
14/47
15/47
16/47
17/47
18/47
19/47
20/47
21/47
22/47
23/47
24/47
25/47
26/47
27/47
28/47
29/47
30/47
31/47
32/47
33/47
34/47
35/47
36/47
37/47
38/47
39/47
40/47
41/47
42/47
43/47
44/47
45/47
46/47
47/47
Level: 2
1/131
2/131
3/131
4/131
5/131
6/131
7/131
8/131
9/131
10/131
11/131
12/131
13/131
14/131
15/131
16/131
17/131
18/131
19/131
20/131
21/131
22/131
23/131
24/131
25/131
26/131
27/131
28/131
29/131
30/131
31/131
32/131
33/131

## Interpreting Results

### Proportion correct

In [19]:
directory_paths = ["/content/drive/My Drive/Research/Multi Agent/MATHByLevel_base/train/number_theory/Level_1",
                  "/content/drive/My Drive/Research/Multi Agent/MATHByLevel_base/train/number_theory/Level_2",
                  "/content/drive/My Drive/Research/Multi Agent/MATHByLevel_base/train/number_theory/Level_3",
                  "/content/drive/My Drive/Research/Multi Agent/MATHByLevel_base/train/number_theory/Level_4",
                  "/content/drive/My Drive/Research/Multi Agent/MATHByLevel_base/train/number_theory/Level_5"
                  ]

count = 0
total = 0

for index, directory_path in enumerate(directory_paths, start=1):

    for dir_name in os.listdir(directory_path):
        dir_path = os.path.join(directory_path, dir_name)

        if os.path.isdir(dir_path):
            directory_data = {'Directory Name': int(dir_name)}

            json_files = [file for file in (os.listdir(dir_path)) if file.endswith('.json')]
            if len(json_files) == 0:
                continue

            json_file = sorted(json_files)[-1]
            json_file_path = os.path.join(dir_path, json_file)

            try:
                with open(json_file_path, 'r') as f:
                    json_data = json.load(f)
                    if json_data['yes_no'] == 'yes':
                        count += 1
                    elif json_data['yes_no'] == 'no':
                        count += 0
                    total += 1
            except FileNotFoundError:
                print("File Not Found")

    print(f'Level {index}: {count}/{total}')

print("\nDone")

Level 1: 40/47
Level 2: 155/178
Level 3: 312/369
Level 4: 431/556
Level 5: 597/869

Done


# Base Cases (MATH) (With Prefix)

## Prompt Prefixes

In [21]:
impersonation = ["Approach the following question the way someone with a PhD in mathematics would. ",
                 "You are an expert in number theory. Solve the following problem. "
                 ]

## Processing

In [22]:
parent_directory = "/content/drive/My Drive/Research/Meeting Prep/6Dataset 1/MATHByLevel/train/number_theory"
output_base_directory = "/content/drive/My Drive/Research/Multi Agent/MATHByLevel_base_with_prefix/train/number_theory"

pattern = r'\\boxed{([^}]*)}'
os.makedirs(output_base_directory, exist_ok=True)

In [20]:
def process_file(input_file_path, output_directory):

    filename = os.path.basename(input_file_path)
    output_directory_name = os.path.splitext(filename)[0]
    output_directory = os.path.join(output_directory, output_directory_name)
    os.makedirs(output_directory, exist_ok=True)

    json_files_count = len([filename for filename in os.listdir(output_directory) if filename.endswith(".json")])
    if json_files_count == len(impersonation):
        print(f"Skipping {output_directory} (already complete)")
        return

    with open(input_file_path, 'r') as file:
        data = json.load(file)

        problem_prompt = data.get('problem')
        solution = data.get('solution')

        for prefix in impersonation:
            problem_prompt_with_prefix = prefix + problem_prompt

            match = re.search(pattern, solution)

            if match:
                for i in range(num_attempts):
                    attempts = i + 1
                    gpt_response = chat_gpt(problem_prompt_with_prefix)
                    solution_value = match.group(1)
                    solution_prompt = "The following is a response to a question. Yes or no, is the final solution obtained in the following response " + str(solution_value) + "?: " + gpt_response.choices[0].message.content.strip()
                    gpt_response_correct_incorrect = chat_gpt(solution_prompt)
                    yes_no = get_yes_no(gpt_response_correct_incorrect)
                    if yes_no == 'yes':
                        break;

                name = prefix[:prefix.rfind(".")]
                output_filename = f"{name}.json"
                output_file_path = os.path.join(output_directory, output_filename)
                with open(output_file_path, 'w') as output_file:
                    json.dump({
                        'prefix': prefix,
                        'problem_prompt': problem_prompt_with_prefix,
                        'solution': solution,
                        'gpt_response': str(gpt_response.choices[0].message.content.strip()),
                        'solution_prompt': solution_prompt,
                        'gpt_response_correct_incorrect': str(gpt_response_correct_incorrect.choices[0].message.content.strip()),
                        'attempts': attempts,
                        'yes_no': yes_no
                    }, output_file, indent=4)

            else:
                print("No text inside \\boxed{} found in the solution string:", output_directory)

def process_directory(input_directory, output_base_directory):
    output_directory = os.path.join(output_base_directory, os.path.basename(input_directory))
    os.makedirs(output_directory, exist_ok=True)

    counter = 1
    for filename in os.listdir(input_directory):
        if filename.endswith(".json"):
            input_file_path = os.path.join(input_directory, filename)
            process_file(input_file_path, output_directory)
            print(f'{counter}/{len(os.listdir(input_directory))}')
            counter += 1

In [30]:
num_attempts = 3
level = 1

for directory in os.listdir(parent_directory):
    input_directory = os.path.join(parent_directory, directory)

    print("Level:", level)
    if os.path.isdir(input_directory):
        process_directory(input_directory, output_base_directory)

    level += 1

print("Done")

Level: 1
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_base_with_prefix/train/number_theory/Level_1/507 (already complete)
1/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_base_with_prefix/train/number_theory/Level_1/1054 (already complete)
2/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_base_with_prefix/train/number_theory/Level_1/446 (already complete)
3/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_base_with_prefix/train/number_theory/Level_1/15 (already complete)
4/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_base_with_prefix/train/number_theory/Level_1/347 (already complete)
5/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_base_with_prefix/train/number_theory/Level_1/49 (already complete)
6/47
Skipping /content/drive/My Drive/Research/Multi Agent/MATHByLevel_base_with_prefix/train/number_theory/Level_1/722 (already complete)
7/47
Skipping /content/d

## Interpreting Results

### Generate CSV

In [32]:
directory_path = "/content/drive/My Drive/Research/Multi Agent/MATHByLevel_base_with_prefix/train/number_theory/Level_5"
csv_file_path = "/content/drive/My Drive/Research/Multi Agent/level_5_number_theory_MATHByLevel_base_with_prefix_data.csv"

data = []

json_files = ["Approach the following question the way someone with a PhD in mathematics would.json",
              "You are an expert in number theory. Solve the following problem.json"
              ]

for dir_name in os.listdir(directory_path):
    dir_path = os.path.join(directory_path, dir_name)

    if os.path.isdir(dir_path):
        directory_data = {'Directory Name': int(dir_name)}

        for json_file in json_files:
            json_file_path = os.path.join(dir_path, json_file)
            try:
                with open(json_file_path, 'r') as f:
                    json_data = json.load(f)
                    directory_data[json_file] = json_data['yes_no']
            except FileNotFoundError:
                directory_data[json_file] = 'N/A'

        data.append(directory_data)

sorted_data = sorted(data, key=lambda x: x['Directory Name'])

with open(csv_file_path, 'w', newline='') as csvfile:
    fieldnames = ['Directory Name'] + json_files
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for row in sorted_data:
        writer.writerow(row)

print("Done:", csv_file_path)

Done: /content/drive/My Drive/Research/Multi Agent/level_5_number_theory_MATHByLevel_base_with_prefix_data.csv


### Analyze CSV

In [33]:
csv_file_paths = ["/content/drive/My Drive/Research/Multi Agent/level_1_number_theory_MATHByLevel_base_with_prefix_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/level_2_number_theory_MATHByLevel_base_with_prefix_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/level_3_number_theory_MATHByLevel_base_with_prefix_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/level_4_number_theory_MATHByLevel_base_with_prefix_data.csv",
                  "/content/drive/My Drive/Research/Multi Agent/level_5_number_theory_MATHByLevel_base_with_prefix_data.csv"
                  ]

for csv_file_path in csv_file_paths:
    print(csv_file_path)

    unique_combinations = {}

    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)

        for row in reader:
            row_values = row[1:]

            combination = ''.join([val[:1] for val in row_values])

            unique_combinations[combination] = unique_combinations.get(combination, 0) + 1

    table_data = []
    total_count = 0
    for combination, count in unique_combinations.items():
        PhD_mathematics = combination[0]
        Expert_number_theory = combination[1]
        table_data.append([PhD_mathematics, Expert_number_theory, count])
        total_count += count

    table_data.append(["Total", "", total_count])

    print(tabulate(table_data, headers=['PhD_mathematics', 'Expert_number_theory', 'Count']))
    print("-------------------------------------------------\n")

/content/drive/My Drive/Research/Multi Agent/level_1_number_theory_MATHByLevel_base_with_prefix_data.csv
PhD_mathematics    Expert_number_theory      Count
-----------------  ----------------------  -------
n                  y                             3
y                  y                            40
y                  n                             4
Total                                           47
-------------------------------------------------

/content/drive/My Drive/Research/Multi Agent/level_2_number_theory_MATHByLevel_base_with_prefix_data.csv
PhD_mathematics    Expert_number_theory      Count
-----------------  ----------------------  -------
y                  y                           100
y                  n                            20
n                  y                             5
n                  n                             6
Total                                          131
-------------------------------------------------

/content/drive/My Drive/R